In [2]:
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

# Generate a dummy regression problem with 2 outputs
X, y = make_regression(n_samples=100, n_features=1, n_targets=2, noise=0.1)
df = pd.DataFrame(X, columns=['X'])
df['y1'] = y[:, 0]
df['y2'] = y[:, 1]

# Train - test split
X_train, X_test, y_train, y_test = train_test_split(df[['X']], df[['y1', 'y2']], test_size=0.2, random_state=42)
print(f'Training set: {X_train.shape[0]} samples')
print(f'Test set: {X_test.shape[0]} samples')

Training set: 80 samples
Test set: 20 samples


In [3]:
from sklearn.linear_model import LinearRegression
model = LinearRegression().fit(X_train, y_train)
print(f'R^2: {model.score(X_test, y_test)}')

R^2: 0.9999964810647388


In [4]:
import numpy as np
import torch
import torch.nn as nn

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

def np_to_ts(x):
    n_samples = len(x)
    return torch.from_numpy(x).to(torch.float).to(DEVICE).reshape(n_samples, -1)

class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        
        self.fc = nn.Linear(self.input_size, 10)
        self.relu = nn.ReLU()
        self.out = nn.Linear(10, self.output_size)
        
    def forward(self, x):
        x = self.fc(x)
        x = self.relu(x)
        x = self.out(x)
        return x

In [5]:
# Train the model
model = Net(1, 2).to(DEVICE)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

X_train_ts = np_to_ts(X_train.values)
y_train_ts = np_to_ts(y_train.values)

EPOCHS = 10000
for epoch in range(EPOCHS):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_ts)
    loss = criterion(outputs, y_train_ts)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, loss {loss.item()}')
        
# Evaluate the model
X_test_ts = np_to_ts(X_test.values)
y_test_ts = np_to_ts(y_test.values)
with torch.no_grad():
    y_pred = model(X_test_ts)
    loss = criterion(y_pred, y_test_ts)
    print(f'Loss: {loss.item()}')

Epoch 0, loss 3866.430419921875
Epoch 100, loss 2259.97314453125
Epoch 200, loss 585.9000854492188
Epoch 300, loss 118.8823471069336
Epoch 400, loss 52.86066436767578
Epoch 500, loss 45.09247589111328
Epoch 600, loss 41.52095031738281
Epoch 700, loss 37.90437698364258
Epoch 800, loss 34.22755813598633
Epoch 900, loss 29.83445167541504
Epoch 1000, loss 24.86443328857422
Epoch 1100, loss 19.976909637451172
Epoch 1200, loss 16.033733367919922
Epoch 1300, loss 12.892077445983887
Epoch 1400, loss 10.459759712219238
Epoch 1500, loss 8.677881240844727
Epoch 1600, loss 7.149247646331787
Epoch 1700, loss 5.9136128425598145
Epoch 1800, loss 4.972369194030762
Epoch 1900, loss 4.27785587310791
Epoch 2000, loss 3.7392711639404297
Epoch 2100, loss 3.283195972442627
Epoch 2200, loss 2.873563766479492
Epoch 2300, loss 2.4874534606933594
Epoch 2400, loss 2.165773630142212
Epoch 2500, loss 1.8796215057373047
Epoch 2600, loss 1.5835232734680176
Epoch 2700, loss 1.3187429904937744
Epoch 2800, loss 1.08984

In [9]:
y_pred = model(X_test_ts).cpu().detach().numpy()

from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.9999964197664062

In [28]:
results_df = pd.DataFrame({'y1_true': y_test['y1'], 'y1_pred': y_pred[:, 0], 'y2_true': y_test['y2'], 'y2_pred': y_pred[:, 1]})
results_df.head()

,y1_true,y1_pred,y2_true,y2_pred
83,-1.889057,-1.896921,-0.982847,-1.106185
53,-79.251498,-79.185402,-45.977589,-46.008533
70,-53.650165,-53.737904,-31.317897,-31.224285
45,-160.601376,-160.496933,-93.384369,-93.248161
44,102.936410,102.961617,59.808890,59.845436
